In [1]:
import numpy as np

# Parallel computation using numba
from numba import jit, njit, prange
from numba import cuda

import warnings

warnings.filterwarnings("ignore")

In [2]:
i = complex(0, 1)
# To be used in the Heston pricer


@jit
def fHeston(s, St, K, r, T, sigma, kappa, theta, volvol, rho):
    # To be used a lot
    prod = rho * sigma * i * s

    # Calculate d
    d1 = (prod - kappa) ** 2
    d2 = (sigma**2) * (i * s + s**2)
    d = np.sqrt(d1 + d2)

    # Calculate g
    g1 = kappa - prod - d
    g2 = kappa - prod + d
    g = g1 / g2

    # Calculate first exponential
    exp1 = np.exp(np.log(St) * i * s) * np.exp(i * s * r * T)
    exp2 = 1 - g * np.exp(-d * T)
    exp3 = 1 - g
    mainExp1 = exp1 * np.power(exp2 / exp3, -2 * theta * kappa / (sigma**2))

    # Calculate second exponential
    exp4 = theta * kappa * T / (sigma**2)
    exp5 = volvol / (sigma**2)
    exp6 = (1 - np.exp(-d * T)) / (1 - g * np.exp(-d * T))
    mainExp2 = np.exp((exp4 * g1) + (exp5 * g1 * exp6))

    return mainExp1 * mainExp2


# Heston Pricer (allow for parallel processing with numba)


@jit(forceobj=True)
def priceHestonMid(St, K, r, T, sigma, kappa, theta, volvol, rho):
    P, iterations, maxNumber = 0, 1000, 100
    ds = maxNumber / iterations

    element1 = 0.5 * (St - K * np.exp(-r * T))

    # Calculate the complex integral
    # Using j instead of i to avoid confusion
    for j in prange(1, iterations):
        s1 = ds * (2 * j + 1) / 2
        s2 = s1 - i

        numerator1 = fHeston(s2, St, K, r, T, sigma, kappa, theta, volvol, rho)
        numerator2 = K * fHeston(s1, St, K, r, T, sigma, kappa, theta, volvol, rho)
        denominator = np.exp(np.log(K) * i * s1) * i * s1

        P = P + ds * (numerator1 - numerator2) / denominator

    element2 = P / np.pi

    return np.real((element1 + element2))

In [25]:
from bs4 import BeautifulSoup
import re
import time 
import pandas as pd
import os
from datetime import datetime as dt
import time
import itertools
# Selenium Related packages
###############################
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url = 'https://www.barchart.com/stocks/quotes/$SPX/options'
driver = webdriver.Chrome()
driver.get(url)
time.sleep(2)
# Accept terms and conditions
driver.find_elements(By.XPATH, "//*[contains(text(), 'Accept all')]")[0].click()
# We're using the barchart page to extract options quotes
rows = Select(driver.find_element(By.XPATH, "//select[@name = 'Moneyness')]"))
rows.select_by_value('inTheMoney')
# Select a date on the options market
mButtons =Select(driver.find_element(By.XPATH, "//select[@data-event-name = 'onExpirationDateChanged']"))
surface =  []
dateIndex = []
for option in quotes.options:
    # Ignore quotes we can't find
    try:
        option.click()
        # Wait for page to load
        time.sleep(5)
        # Find the call and put tables
        #Select first table (Call)
        tableDiv = option.find_elements_by_xpath(
              "//div[@class = 'bc-table-scrollable-inner']")[0]
        # We want the first table(Call quotes)
        callQuotes = tableDiv.find_elements_by_xpath("//tbody")[0]
        callQuotes = callQuotes.find_elements_by_xpath(
                     "//tr[@sly-repeat = 'row in rows.data']")
        # We want to obtain all the quotes from each row
        regex = '[0-9,.]{2,}'
        quotes = [re.findall(regex, elem.text) 
                   for elem in callQuotes]
        #Strikes are element 0, mid quotes element 4 on table
        strikes = [float(re.sub(',','', quote[0])) for quote in quotes]
        midQuotes = [float(re.sub(',','', quote[3])) for quote in quotes]
        # Construct the dataframe
        date = re.search('[0-9]{4}-[0-9]{2}-[0-9]{2}', option.text).group()
        date = dt.strptime(date, '%Y-%m-%d')
        # Create the date 
        surface.append(midQuotes)
        dateIndex.append(date)
    except Exception:
        pass

# Construct a full volatility surface
maturities = [(date - dt.today()).days/365.25  for date in dateIndex]
volSurface = pd.DataFrame(surface, index = maturities, columns = strikes)
volSurface = volSurface[volSurface.index > 0.1]
volSurface

InvalidSelectorException: Message: invalid selector: Unable to locate an element with the xpath expression //[@name = 'Moneyness')] because of the following error:
SyntaxError: Failed to execute 'evaluate' on 'Document': The string '//[@name = 'Moneyness')]' is not a valid XPath expression.
  (Session info: chrome=119.0.6045.107); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalid-selector-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7409982B2+55298]
	(No symbol) [0x00007FF740905E02]
	(No symbol) [0x00007FF7407C05AB]
	(No symbol) [0x00007FF7407C509C]
	(No symbol) [0x00007FF7407C6FDF]
	(No symbol) [0x00007FF7407C70D0]
	(No symbol) [0x00007FF7408013E4]
	(No symbol) [0x00007FF7408018DC]
	(No symbol) [0x00007FF74083CBC7]
	(No symbol) [0x00007FF7408220EF]
	(No symbol) [0x00007FF74083AAA4]
	(No symbol) [0x00007FF740821E83]
	(No symbol) [0x00007FF7407F670A]
	(No symbol) [0x00007FF7407F7964]
	GetHandleVerifier [0